# なんですかこれは
- できるだけ小さい問題設定・プログラムで探索アルゴリズムを実装してみる
- アルゴリズム以外の部分（判定ロジックとか）を極力削ぎ落とすことを目指して作っている
- 更新
    - 2021.05.13 A*探索まで書いた
    - 2021.05.14 heapq の説明をちょっと修正

---

# キュー操作用標準ライブラリについて
## collections.deque
- list.pop(0) は遅い
    - 計算量は $O(N)$
- 代わりに deque.popleft() を使うべし
    - 計算量は $O(1)$
- ちなみに queue.Queue というのもあるが、基本的には deque 一択
    - deque の方が高速
    - 左右両側から push, pop できる
        - deque = Double-Ended Queue
    - queue ライブラリはスレッドセーフ：マルチスレッド時に有力

In [1]:
from collections import deque

queue = deque([0, 1, 2])    # キューの作成
print(queue, type(queue))

queue.append(3)             # 追加
print(queue)

print(queue.popleft())      # 左から pop
print(queue)

deque([0, 1, 2]) <class 'collections.deque'>
deque([0, 1, 2, 3])
0
deque([1, 2, 3])


## heapq
- ヒープソートされている list を優先度付きキューとして扱える便利な関数群
    - ヒープソート：pop を高速にできるように並び替えること（完全なソートではない）
    - 未ソートなら heapq.heapify() でヒープソートできる
- キューは (key, value) のような tuple の list として表現する
    - あくまで扱うのは list
    - 内部では tuple 同士の比較によってソートしているだけ
- heapq.heappop() で key が最小のものから優先的に取り出される
- heapq.heappush() は追加と同時にヒープソートするようなもの

In [2]:
from heapq import heapify, heappush, heappop

queue = [(5, "a"), (7, "b"), (1, "c"), (3, "d")]
print("初期状態：", queue)

# 優先度付きキューにする（⇔ ヒープソートする）
heapify(queue)
print("ソート後：", queue, type(queue))

# push
heappush(queue, (4, "e"))
print("push後：", queue)

# pop
print("pop:", heappop(queue))
print("pop後：", queue)

初期状態： [(5, 'a'), (7, 'b'), (1, 'c'), (3, 'd')]
ソート後： [(1, 'c'), (3, 'd'), (5, 'a'), (7, 'b')] <class 'list'>
push後： [(1, 'c'), (3, 'd'), (5, 'a'), (7, 'b'), (4, 'e')]
pop: (1, 'c')
pop後： [(3, 'd'), (4, 'e'), (5, 'a'), (7, 'b')]


---

# 幅優先探索 Breadth First Search
- 環境
    - 4 つのノード 0, 1, 2, 3 がある
    - グラフが隣接行列の疎行列表現 GRAPH で与えられる
        - GRAPH\[i\] はノード i の子ノード（隣接ノード）のリスト
        - グラフは $ 0 - 1 \lhd_3^2 $ みたいな形
    - スタートノード START は 0

- お約束
    - 幅優先探索における経路コストの定義
        - スタートからのノード数のこと
    - 子ノードの定義
        - ノードと隣接するノードのこと

- アルゴリズム
    - 概要
        - あるノードに対して、スタートからの最短経路を求めるアルゴリズム
        - ここでは全てのノードについて経路コストを計算することを目指す
            - 経路を求めるにはちょっとコードが嵩張るのでここではパス
            - ゴールを求める問題なら途中で打ち切ればいいだけ
                - 全てのノードが見つかるならゴールも見つかるので
    - 【下準備】
        - 経路コストのリスト costs を用意
            - costs\[i\] はノード i の経路コスト
            - ついでに初期値 -1 で未確定を表現すると、子ノードを処理する条件が判定しやすい
                - 「経路コストが確定している ⇔ 訪問済みまたは訪問予定である」から
                - 対偶を取ると、「経路コストが未確定 ⇔ 未訪問かつ訪問予定でない」
            - スタートノードの経路コストは 0
        - 訪問予定のノードのキュー queue を用意
            - 最初はスタートノードを訪問する
    - 【探索】queue が空になるまで順にノード node を取り出す（訪問する）：
        - node の各子ノード child に対して、未訪問かつ訪問予定でないなら：
            - 子ノードの経路コストを計算：現在のノードの経路コスト + 1
            - child を queue に追加

- 補足
    - この問題は実質無向グラフだが、幅優先探索は一般の有向グラフに適用できる
    - 訪問済みかどうかの判定について
        - 巡回路がない場合は無くても動くっちゃ動く
        - が、基本的には無限ループの原因になる
        - また訪問した時点で既に距離は確定しており、複数回の訪問は無駄である
    - 経路を調べたければ parent を記録する必要がある
        - class を使ってノードの値と一緒にまとめると楽 → プロ実のような構成になる

In [3]:
""" 環境定義 """
GRAPH = [[1], [0, 2, 3], [1, 3], [1, 2]]
START = 0

""" 幅優先探索 """
costs = [0, -1, -1, -1] # costs[i] はノード i の経路コスト
queue = deque([START])  # 訪問予定のノードのキュー

while queue:
    node = queue.popleft()  # 訪問：キューから取り出す
    print(f"ノード {node} の距離は {costs[node]}")  # ゴール探しならここで判定して break すればいい

    for child in GRAPH[node]:               # 各子ノードに対して
        if costs[child] == -1:              # 未訪問かつ訪問予定でない（⇔ 経路コストが -1）なら
            costs[child] = costs[node] + 1  # 子ノードの経路コストを計算
            queue.append(child)             # 訪問予定キューに追加

print(costs)    # 距離のリスト

ノード 0 の距離は 0
ノード 1 の距離は 1
ノード 2 の距離は 2
ノード 3 の距離は 2
[0, 1, 2, 2]


# Dijkstra アルゴリズム
- 概要
    - ノード間にコストが定義された場合の幅優先探索
    - 経路コストはそれをスタートから合計したものと定義される
        - ノード $n$ に対し、$g(n)$ で表すのが慣例
    - 経路コストが小さいものから優先的に探索する
    - 最適解を最初に発見できる

- 環境
    - https://qiita.com/shizuma/items/e08a76ab26073b21c207 のグラフを使う  
        ![](https://camo.qiitausercontent.com/645ed43161c6afc7ac70094fd2003c472c396fd3/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f34303739362f65633738343563632d653038632d653763342d666638612d3664656233306236363732382e706e67)
    - ノード間のコストは隣接行列のコスト版みたいなやつ NODE_COSTS で与えられる
    - スタートノードとゴールノードをそれぞれ 1, 5 とする
        - プログラムでは START, GOAL = 0, 4
    - 最短経路は 1 -> 2 -> 3 -> 5
    - その経路コストは 85

- アルゴリズム
    - 【下準備】
        - 暫定的な経路コストのリスト costs を用意
            - 10000 とか十分大きな値で初期化しとくと後で楽
            - スタートノードの経路コストは 0
        - 訪問予定のノードのキュー queue を用意
            - 最初はスタートノードを訪問する
        - 訪問済みノードのリスト visited を用意
            - 今回 costs は暫定的な経路コストとして扱うため、訪問済み判定には使えない
            - ちなみに in 演算子での比較は list より set の方が高速
    - 【探索】queue が空になるまで順にノード node を取り出す（訪問する）：
        - まずは node を訪問済みリストに追加
        - node の各子ノード child に対して、未訪問なら：
            - 子ノードの経路コストを計算：node の距離 + node-child 間のコスト
                - 以前の暫定経路コストより短ければ更新
            - child を queue に追加

In [4]:
""" 環境定義 """
NODE_COSTS = [
    [ 0, 50, 80,  0,  0],
    [ 0,  0, 20, 15,  0],
    [ 0,  0,  0, 10, 15],
    [ 0,  0,  0,  0, 30],
    [ 0,  0,  0,  0,  0]
]
N = len(NODE_COSTS) # ノード数
START, GOAL = 0, 4

In [5]:
# node の子ノードのジェネレータ
def children(node: int) -> int:
    for child in range(N):
        if NODE_COSTS[node][child]:
            yield child

In [6]:
""" Dijkstra アルゴリズム """
costs = [10000] * N
costs[START] = 0
queue = [(0, START)]
visited = set()             # 訪問済みリスト

while queue:
    node = heappop(queue)[1]
    visited.add(node)       # 訪問済みリストに追加
    print(f"ノード {node+1} の距離は {costs[node]}")
    if node == GOAL:
        print("ゴール")

    for child in children(node):    # 各子ノードに対して
        if child not in visited:    # 未訪問なら

            # 子ノードの経路コストを計算
            costs[child] = min(costs[child], costs[node] + NODE_COSTS[node][child])

            if all(v!=child for _, v in queue):         # 訪問予定でなければ
                heappush(queue, (costs[child], child))  # 訪問予定キューに追加

ノード 1 の距離は 0
ノード 2 の距離は 50
ノード 4 の距離は 65
ノード 3 の距離は 70
ノード 5 の距離は 85
ゴール


# 最良優先探索 Best First Search
- 概要
    - Dijkstra アルゴリズムの優先条件を一般の評価関数にしたもの：
        - ノードを評価する関数を用意し、評価の高いノードから優先的に探索する
        - Dijkstra アルゴリズムは最良優先探索の一種
    - 最適解を最初に発見できるとは限らない

- 環境
    - Dijkstra アルゴリズムと同じ
    - ただし、今回は神様協力のもと各ノードでそのノードを経由した時の最小の経路コストが分かるとする

- 評価関数 evaluate()
    - ノード経由時の経路コストが分かるので、ここではそれを評価値として定義する
        - この値が小さいノードから優先的に探索する

In [31]:
def evaluate(node: int) -> int:
    return [85, 85, 85, 95, 85][node]

In [33]:
costs = [10000] * N
costs[START] = 0
queue = [(evaluate(START), START)]
visited = set()

while queue:
    node = heappop(queue)[1]
    visited.add(node)
    print(f"ノード {node+1} の経路コストは {costs[node]}、評価値は {evaluate(node)}")
    if node == GOAL:
        print("ゴール")

    for child in children(node):
        if child not in visited:
            costs[child] = min(costs[child], costs[node] + NODE_COSTS[node][child])
            if all(v!=child for _, v in queue):
                heappush(queue, (evaluate(child), child))   # 優先度を評価値で定義

ノード 1 の経路コストは 0、評価値は 85
ノード 2 の経路コストは 50、評価値は 85
ノード 3 の経路コストは 70、評価値は 85
ノード 5 の経路コストは 85、評価値は 85
ゴール
ノード 4 の経路コストは 65、評価値は 95


# A* 探索
- 神様なんていないだろ！いい加減にしろ！
    - （ ˘⊖˘）。o(待てよ？じゃあ推定すればよくね？)
    - |ユークリッド空間|┗(☋｀ )┓三
    - ( ◠‿◠ )☛ A* 探索

- 評価関数
    - 急いでる人はこちら
        - ノード n の評価値を $ f(n) = g(n) + h(n) $ で定義する
        - ただし $g(n)$ は n の経路コスト、$h(n)$ は n から GOAL までのコストの推定値
    - 要するに n を経由した場合の GOAL の経路コストの推定値
    - 肝心の $h(n)$ はどうやって推定すんだよ
        - 迷路なら n, GOAL 間のユークリッド距離やマンハッタン距離などで定義する
        - ここでは GOAL までの最小ノード数を10倍した値で定義する
        - ちなみに $h(n)$ をヒューリスティック関数という
        - $h(n)$ が許容的である（⇔ $h(n)$ が実際の値以下である）ならば、最適解を最初に発見する
            - 参考）http://kussharo.complex.eng.hokudai.ac.jp/~kurihara/classes/AI/heuristic.pdf

In [50]:
def astar_eval(node: int, costs: int) -> int:
    return costs[node] + 10 * [2, 2, 1, 1, 0][node]  # g + h

In [51]:
costs = [10000] * N
costs[START] = 0
queue = [(Astar_eval(START, costs), START)]
visited = set()

while queue:
    node = heappop(queue)[1]
    visited.add(node)
    print(f"ノード {node+1} の経路コストは {costs[node]}、評価値は {astar_eval(node, costs)}")
    if node == GOAL:
        print("ゴール")

    for child in children(node):
        if child not in visited:
            costs[child] = min(costs[child], costs[node] + NODE_COSTS[node][child])
            if all(v!=child for _, v in queue):
                heappush(queue, (astar_eval(child, costs), child))

ノード 1 の経路コストは 0、評価値は 20
ノード 2 の経路コストは 50、評価値は 70
ノード 4 の経路コストは 65、評価値は 75
ノード 3 の経路コストは 70、評価値は 80
ノード 5 の経路コストは 85、評価値は 85
ゴール


# 包含関係
- 幅優先探索
    - 最良優先探索（評価関数の値が高いノードを優先的に探索するようにしたもの）
        - A* 探索（評価関数を (その時点の経路コスト $g$ + そこからゴールまでの合計コスト $h$) で定義したもの）
            - Dijkstra 法（$h$ を定数関数にしたもの）